In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt

local_dir = "./graphing_eval_results/"
image_path = "./temp_images"

In [ ]:
import itertools
import os

from sae_bench_utils.sae_selection_utils import select_saes_multiple_patterns
import sae_bench_utils.graphing_utils as graphing_utils

selections = {
    # "Gemma-Scope Gemma-2-2B Width Series": [
    #     [
    #         r"(gemma-scope-2b-pt-res)",
    #     ],
    #     [
    #         r".*layer_{layer}.*(16k|65k|1m).*",
    #     ],
    # ],
    # "Gemma-Scope Gemma-2-9B Width Series": [
    #     [
    #         r"(gemma-scope-9b-pt-res)",
    #     ],
    #     [
    #         r".*layer_{layer}.*(16k|131k|1m).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B TopK Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_topk_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow16_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B Vanilla Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B 4K Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow12_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B 16K Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow14_date-1109",
    #         r"(gemma-scope-2b-pt-res)",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*layer_{layer}.*(16k).*",
    #     ],
    # ],
    "SAE Bench Gemma-2-2B 65K Width Series": [
        [
            r"sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109",
            r"sae_bench_gemma-2-2b_topk_width-2pow16_date-1109",
            r"(gemma-scope-2b-pt-res)",
        ],
        [
            r".*blocks\.{layer}(?!.*step).*",
            r".*blocks\.{layer}(?!.*step).*",
            r".*layer_{layer}.*(65k).*",
        ],
    ],
    # "SAE Bench Pythia-70M SAE Type Series": [
    #     [
    #         r"sae_bench_pythia70m_sweep.*_ctx128_.*",
    #     ],
    #     [r".*blocks\.({layer})\.hook_resid_post__trainer_.*"],
    # ],
}

eval_path = f"{local_dir}/unlearning"
eval_path = f"{local_dir}/scr"
eval_path = f"{local_dir}/sparse_probing"
# eval_path = f"{local_dir}/tpp"
# eval_path = f"{local_dir}/absorption"

eval_paths = [
    # f"{local_dir}/sparse_probing",
    # f"{local_dir}/tpp",
    f"{local_dir}/scr",
    f"{local_dir}/core",
    # f"{local_dir}/absorption",
    f"{local_dir}/autointerp",
    f"{local_dir}/unlearning",
]

core_results_path = f"{local_dir}/core"

combinations = list(itertools.product(eval_paths, selections.keys()))

ks_lookup = {
    "scr": [50],
    "tpp": [50],
    "sparse_probing": [1],
}

figures = []

for eval_path, selection in combinations:
    eval_type = eval_path.split("/")[-1]

    if eval_type in ks_lookup:
        ks = ks_lookup[eval_type]
    else:
        ks = [-1]

    if "Gemma-2-2B" in selection:
        layers = [12]
    # elif "Pythia-70M" in selection:
    #     layers = [3, 4]
    # elif "Gemma-2-9B" in selection:
    #     layers = [9, 20, 31]
    else:
        raise ValueError("Please add the correct layers for the selection")

    layer_ks_combinations = list(itertools.product(layers, ks))

    for layer, k in layer_ks_combinations:
        sae_regex_patterns = selections[selection][0]
        sae_block_patterns = list(selections[selection][1])

        for i, pattern in enumerate(sae_block_patterns):
            sae_block_patterns[i] = pattern.format(layer=layer)

        suptitle = f"{selection} Layer {layer}\n"
        prefix = ""

        image_base_folder = os.path.join(image_path, eval_type)

        if not os.path.exists(image_base_folder):
            os.makedirs(image_base_folder)

        image_base_name = os.path.join(
            image_base_folder, f"{selection.replace(' ', '_').lower()}_layer_{layer}"
        )

        if eval_type in ks_lookup:
            image_base_name = f"{image_base_name}_topk_{k}"

        selected_saes = select_saes_multiple_patterns(sae_regex_patterns, sae_block_patterns)

        fig = graphing_utils.get_single_figure(
            selected_saes,
            eval_path,
            core_results_path,
            image_base_name,
            k=k,
            title=prefix,
        )
        figures.append(fig)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Add a main title
fig.suptitle(suptitle, fontsize=20, y=0.92)

# Adjust the spacing between subplots
plt.subplots_adjust(
    wspace=0.1,  # Width spacing between subplots
    hspace=0.0,  # Height spacing between subplots
    left=0.05,   # Left margin
    right=0.9,  # Right margin
    top=0.90,    # Top margin - adjusted to make room for suptitle
    bottom=0.05  # Bottom margin
)


for ax, sub_fig in zip(axes.flatten(), figures):
    # Draw each figure's content onto the subplot axes
    sub_fig.canvas.draw()
    ax.imshow(sub_fig.canvas.buffer_rgba())
    ax.axis('off')  # Optional: Turn off axis for clarity

plt.tight_layout()
plt.savefig(f"2x2_plot.png")
plt.show()

In [ ]:
import itertools
import os

from sae_bench_utils.sae_selection_utils import select_saes_multiple_patterns
import sae_bench_utils.graphing_utils as graphing_utils

selections = {
    "Gemma-Scope Gemma-2-2B Width Series": [
        [
            r"(gemma-scope-2b-pt-res)",
        ],
        [
            r".*layer_{layer}.*(16k|65k|1m).*",
        ],
    ],
    # "Gemma-Scope Gemma-2-9B Width Series": [
    #     [
    #         r"(gemma-scope-9b-pt-res)",
    #     ],
    #     [
    #         r".*layer_{layer}.*(16k|131k|1m).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B TopK Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_topk_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow16_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B Vanilla Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B 4K Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow12_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B 16K Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow14_date-1109",
    #         r"(gemma-scope-2b-pt-res)",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*layer_{layer}.*(16k).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B 65K Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow16_date-1109",
    #         r"(gemma-scope-2b-pt-res)",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*layer_{layer}.*(65k).*",
    #     ],
    # ],
    # "SAE Bench Pythia-70M SAE Type Series": [
    #     [
    #         r"sae_bench_pythia70m_sweep.*_ctx128_.*",
    #     ],
    #     [r".*blocks\.({layer})\.hook_resid_post__trainer_.*"],
    # ],
}

eval_path = f"{local_dir}/unlearning"
eval_path = f"{local_dir}/scr"
eval_path = f"{local_dir}/sparse_probing"
# eval_path = f"{local_dir}/tpp"
# eval_path = f"{local_dir}/absorption"

eval_paths = [
    f"{local_dir}/sparse_probing",
    # f"{local_dir}/tpp",
    # f"{local_dir}/scr",
    # f"{local_dir}/core",
    # f"{local_dir}/absorption",
    # f"{local_dir}/autointerp",
    # f"{local_dir}/unlearning",
]

core_results_path = f"{local_dir}/core"

combinations = list(itertools.product(eval_paths, selections.keys()))

ks_lookup = {
    "scr": [50],
    "tpp": [50],
    "sparse_probing": [1],
}

figures = []

for eval_path, selection in combinations:
    eval_type = eval_path.split("/")[-1]

    if eval_type in ks_lookup:
        ks = ks_lookup[eval_type]
    else:
        ks = [-1]

    if "Gemma-2-2B" in selection:
        layers = [5, 19]
    # elif "Pythia-70M" in selection:
    #     layers = [3, 4]
    # elif "Gemma-2-9B" in selection:
    #     layers = [9, 20, 31]
    else:
        raise ValueError("Please add the correct layers for the selection")

    layer_ks_combinations = list(itertools.product(layers, ks))

    for layer, k in layer_ks_combinations:
        sae_regex_patterns = selections[selection][0]
        sae_block_patterns = list(selections[selection][1])

        for i, pattern in enumerate(sae_block_patterns):
            sae_block_patterns[i] = pattern.format(layer=layer)

        suptitle = f"{selection}\nSparse Probing Top {k} Accuracy"

        image_base_folder = os.path.join(image_path, eval_type)

        if not os.path.exists(image_base_folder):
            os.makedirs(image_base_folder)

        image_base_name = os.path.join(
            image_base_folder, f"{selection.replace(' ', '_').lower()}_layer_{layer}"
        )

        if eval_type in ks_lookup:
            image_base_name = f"{image_base_name}_topk_{k}"

        title = f"Layer {layer}"

        selected_saes = select_saes_multiple_patterns(sae_regex_patterns, sae_block_patterns)

        fig = graphing_utils.get_single_figure(
            selected_saes,
            eval_path,
            core_results_path,
            image_base_name,
            k=k,
            title=title,
            plot_type=False,
        )
        figures.append(fig)

fig, axes = plt.subplots(1, 2, figsize=(12, 8))

# Add a main title
fig.suptitle(suptitle, fontsize=20, y=0.75)

# Adjust the spacing between subplots
plt.subplots_adjust(
    wspace=0.1,  # Width spacing between subplots
    hspace=0.0,  # Height spacing between subplots
    left=0.05,   # Left margin
    right=0.9,  # Right margin
    top=0.90,    # Top margin - adjusted to make room for suptitle
    bottom=0.05  # Bottom margin
)


for ax, sub_fig in zip(axes.flatten(), figures):
    # Draw each figure's content onto the subplot axes
    sub_fig.canvas.draw()
    ax.imshow(sub_fig.canvas.buffer_rgba())
    ax.axis('off')  # Optional: Turn off axis for clarity

plt.tight_layout()
plt.savefig(f"1x2_sparse_probing_plot.png")
plt.show()

In [ ]:
import itertools
import os

from sae_bench_utils.sae_selection_utils import select_saes_multiple_patterns
import sae_bench_utils.graphing_utils as graphing_utils

selections = {
    # "Gemma-Scope Gemma-2-2B Width Series": [
    #     [
    #         r"(gemma-scope-2b-pt-res)",
    #     ],
    #     [
    #         r".*layer_{layer}.*(16k|65k|1m).*",
    #     ],
    # ],
    # "Gemma-Scope Gemma-2-9B Width Series": [
    #     [
    #         r"(gemma-scope-9b-pt-res)",
    #     ],
    #     [
    #         r".*layer_{layer}.*(16k|131k|1m).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B TopK Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_topk_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow16_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B Vanilla Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow14_date-1109",
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    # "SAE Bench Gemma-2-2B 4K Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow12_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow12_date-1109",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #     ],
    # ],
    "SAE Bench Gemma-2-2B 16K Width Series": [
        [
            r"sae_bench_gemma-2-2b_vanilla_width-2pow14_date-1109",
            r"sae_bench_gemma-2-2b_topk_width-2pow14_date-1109",
            r"(gemma-scope-2b-pt-res)",
        ],
        [
            r".*blocks\.{layer}(?!.*step).*",
            r".*blocks\.{layer}(?!.*step).*",
            r".*layer_{layer}.*(16k).*",
        ],
    ],
    # "SAE Bench Gemma-2-2B 65K Width Series": [
    #     [
    #         r"sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109",
    #         r"sae_bench_gemma-2-2b_topk_width-2pow16_date-1109",
    #         r"(gemma-scope-2b-pt-res)",
    #     ],
    #     [
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*blocks\.{layer}(?!.*step).*",
    #         r".*layer_{layer}.*(65k).*",
    #     ],
    # ],
    # "SAE Bench Pythia-70M SAE Type Series": [
    #     [
    #         r"sae_bench_pythia70m_sweep.*_ctx128_.*",
    #     ],
    #     [r".*blocks\.({layer})\.hook_resid_post__trainer_.*"],
    # ],
}

eval_path = f"{local_dir}/unlearning"
eval_path = f"{local_dir}/scr"
eval_path = f"{local_dir}/sparse_probing"
# eval_path = f"{local_dir}/tpp"
# eval_path = f"{local_dir}/absorption"

eval_paths = [
    # f"{local_dir}/sparse_probing",
    f"{local_dir}/tpp",
    f"{local_dir}/scr",
    # f"{local_dir}/core",
    # f"{local_dir}/absorption",
    # f"{local_dir}/autointerp",
    # f"{local_dir}/unlearning",
]

core_results_path = f"{local_dir}/core"

combinations = list(itertools.product(eval_paths, selections.keys()))

ks_lookup = {
    "scr": [50],
    "tpp": [50],
    "sparse_probing": [1],
}

figures = []

for eval_path, selection in combinations:
    eval_type = eval_path.split("/")[-1]

    if eval_type in ks_lookup:
        ks = ks_lookup[eval_type]
    else:
        ks = [-1]

    if "Gemma-2-2B" in selection:
        layers = [12]
    # elif "Pythia-70M" in selection:
    #     layers = [3, 4]
    # elif "Gemma-2-9B" in selection:
    #     layers = [9, 20, 31]
    else:
        raise ValueError("Please add the correct layers for the selection")

    layer_ks_combinations = list(itertools.product(layers, ks))

    for layer, k in layer_ks_combinations:
        sae_regex_patterns = selections[selection][0]
        sae_block_patterns = list(selections[selection][1])

        for i, pattern in enumerate(sae_block_patterns):
            sae_block_patterns[i] = pattern.format(layer=layer)

        suptitle = f"{selection} Layer {layer}"

        image_base_folder = os.path.join(image_path, eval_type)

        if not os.path.exists(image_base_folder):
            os.makedirs(image_base_folder)

        image_base_name = os.path.join(
            image_base_folder, f"{selection.replace(' ', '_').lower()}_layer_{layer}"
        )

        if eval_type in ks_lookup:
            image_base_name = f"{image_base_name}_topk_{k}"

        selected_saes = select_saes_multiple_patterns(sae_regex_patterns, sae_block_patterns)

        fig = graphing_utils.get_single_figure(
            selected_saes,
            eval_path,
            core_results_path,
            image_base_name,
            k=k,
        )
        figures.append(fig)

fig, axes = plt.subplots(1, 2, figsize=(12, 8))

# Add a main title
fig.suptitle(suptitle, fontsize=20, y=0.75)

# Adjust the spacing between subplots
plt.subplots_adjust(
    wspace=0.1,  # Width spacing between subplots
    hspace=0.0,  # Height spacing between subplots
    left=0.05,   # Left margin
    right=0.9,  # Right margin
    top=0.90,    # Top margin - adjusted to make room for suptitle
    bottom=0.05  # Bottom margin
)


for ax, sub_fig in zip(axes.flatten(), figures):
    # Draw each figure's content onto the subplot axes
    sub_fig.canvas.draw()
    ax.imshow(sub_fig.canvas.buffer_rgba())
    ax.axis('off')  # Optional: Turn off axis for clarity

plt.tight_layout()
plt.savefig(f"1x2_scr_tpp.png")
plt.show()